In [ ]:
from Functions_For_LIF_analysis import import_lif_file, view_2d_images, count_dapi, pixel_intensity_calc
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [ ]:
data_2D = import_lif_file(r"data\RBMC029 epi dif 2D.lif")

In [ ]:
view_2d_images(data_2D )

In [ ]:
cell_counts = {}

for image_obj in data_2D:
    image_name = image_obj['image_name']
    first_channel_images = image_obj['channel_images'][0]

    for frame_index, frame in enumerate(first_channel_images):
        frame_name = f"{image_name}_frame_{frame_index + 1}"
        result = count_dapi(frame_name, frame)

        cell_counts.update(result)

print(cell_counts)

In [ ]:
mean_intensities = {}

for image_obj in data_2D:
    image_name = image_obj['image_name']
    first_channel_images = image_obj['channel_images'][0]

    for frame_index, frame in enumerate(first_channel_images):
        frame_name = f"{image_name}_frame_{frame_index + 1}"
        result = pixel_intensity_calc(frame_name, frame)

        mean_intensities.update(result)

print(mean_intensities)

In [ ]:
normalized_intensities = {}

for image_name, intensity in mean_intensities.items():
    if image_name in cell_counts:
        cell_count = cell_counts[image_name]

        if cell_count != 0:
            normalized_intensity = intensity / cell_count
        else:
            normalized_intensity = 0

        normalized_intensities[image_name] = normalized_intensity

print(normalized_intensities)

In [ ]:
df = pd.DataFrame(list(cell_counts.items()), columns=['Image Name', 'Cell Count'])

df['Intensity'] = df['Image Name'].map(normalized_intensities)

df.sort_values(by='Image Name', inplace=True)

day_1_ndif_avg_intensity = df[df['Image Name'].str.contains('day_1_ndif')]['Intensity'].mean()

df['Normalized Intensity by Day 1'] = df['Intensity'] / day_1_ndif_avg_intensity

df.set_index('Image Name', inplace=True)

df

In [ ]:
#A test to check if the data is normally distributed
groups = {
    'day_1_ndif': 'BMC_day_1_ndif',
    'day_5_dif': 'BMC_day_5_dif',
    'day_5_ndif': 'BMC_day_5_ndif',
    'day_10_dif': 'BMC_day_10_dif',
    'day_10_ndif': 'BMC_day_10_ndif'
}

shapiro_results = {'Group': [], 'Test': [], 'Statistic': [], 'P-Value': []}

for group, pattern in groups.items():
    group_df = df[df.index.str.startswith(pattern)]

    if len(group_df) >= 3:
        stat, p_value = stats.shapiro(group_df['Cell Count'])
        shapiro_results['Group'].append(group)
        shapiro_results['Test'].append('Cell Count')
        shapiro_results['Statistic'].append(stat)
        shapiro_results['P-Value'].append(p_value)

        stat, p_value = stats.shapiro(group_df['Normalized Intensity by Day 1'])
        shapiro_results['Group'].append(group)
        shapiro_results['Test'].append('Normalized Intensity by Day 1')
        shapiro_results['Statistic'].append(stat)
        shapiro_results['P-Value'].append(p_value)
    else:
        print(f"Not enough data points in group {group} for Shapiro-Wilk test.")

shapiro_results_df = pd.DataFrame(shapiro_results)
print(shapiro_results_df)

In [ ]:
#Since the data is normally distributed, I will use the z-test to remove any outliers
groups = {
    'day_1_ndif': 'BMC_day_1_ndif',
    'day_5_dif': 'BMC_day_5_dif',
    'day_5_ndif': 'BMC_day_5_ndif',
    'day_10_dif': 'BMC_day_10_dif',
    'day_10_ndif': 'BMC_day_10_ndif'
}

def filter_outliers(group_df, column):
    z_scores = stats.zscore(group_df[column])
    return group_df[(z_scores < 3) & (z_scores > -3)]

filtered_dfs = []
for group, pattern in groups.items():
    group_df = df[df.index.str.startswith(pattern)]
    
    group_df = filter_outliers(group_df, 'Cell Count')

    group_df = filter_outliers(group_df, 'Normalized Intensity by Day 1')

    filtered_dfs.append(group_df)

filtered_df = pd.concat(filtered_dfs)

filtered_df

In [ ]:
#The t-test is performed to see if there are any statistically significant difference between the groups
group_pairs = {
    ('day_5_ndif', 'day_5_dif'): ('BMC_day_5_ndif', 'BMC_day_5_dif'),
    ('day_10_ndif', 'day_10_dif'): ('BMC_day_10_ndif', 'BMC_day_10_dif')
}

t_test_results = {'Group Pair': [], 'Test Statistic': [], 'P-Value': []}
for pair, (group1_pattern, group2_pattern) in group_pairs.items():
    group1_df = df[df.index.str.startswith(group1_pattern)]
    group2_df = df[df.index.str.startswith(group2_pattern)]

    stat, p_value = stats.ttest_ind(group1_df['Cell Count'], group2_df['Cell Count'], equal_var=True)
    t_test_results['Group Pair'].append(f'{pair[0]} vs {pair[1]} (Cell Count)')
    t_test_results['Test Statistic'].append(stat)
    t_test_results['P-Value'].append(p_value)

    stat, p_value = stats.ttest_ind(group1_df['Normalized Intensity by Day 1'], group2_df['Normalized Intensity by Day 1'], equal_var=True)
    t_test_results['Group Pair'].append(f'{pair[0]} vs {pair[1]} (Normalized Intensity)')
    t_test_results['Test Statistic'].append(stat)
    t_test_results['P-Value'].append(p_value)

t_test_results_df = pd.DataFrame(t_test_results)
print(t_test_results_df)

In [ ]:
groups = ['day_1_ndif', 'day_5_dif', 'day_5_ndif', 'day_10_dif', 'day_10_ndif']

grouped_df = pd.DataFrame(columns=['Cell Count', 'Normalized Intensity by Day 1'])

for group in groups:
    group_data = filtered_df[filtered_df.index.str.contains(group)]
    
    grouped_df.loc[group] = group_data.mean()

print(grouped_df)

In [ ]:
group_patterns = {
    'day_1_ndif': 'BMC_day_1_ndif',
    'day_5_dif': 'BMC_day_5_dif',
    'day_5_ndif': 'BMC_day_5_ndif',
    'day_10_dif': 'BMC_day_10_dif',
    'day_10_ndif': 'BMC_day_10_ndif'
}

error_bar_bounds = []

for group, pattern in group_patterns.items():
    group_data = filtered_df[filtered_df.index.str.contains(pattern)]
    
    if group_data.empty:
        print(f"No data found for group: {group}")
        continue

    mean_cell_count = group_data['Cell Count'].mean()
    std_cell_count = group_data['Cell Count'].std()
    mean_intensity = group_data['Normalized Intensity by Day 1'].mean()
    std_intensity = group_data['Normalized Intensity by Day 1'].std()

    lower_cell_count = mean_cell_count - std_cell_count
    upper_cell_count = mean_cell_count + std_cell_count
    lower_intensity = mean_intensity - std_intensity
    upper_intensity = mean_intensity + std_intensity

    error_bar_bounds.append({'Group': group, 'Measure': 'Lower Bound', 'Cell Count': lower_cell_count, 'Normalized Intensity by Day 1': lower_intensity})
    error_bar_bounds.append({'Group': group, 'Measure': 'Upper Bound', 'Cell Count': upper_cell_count, 'Normalized Intensity by Day 1': upper_intensity})

error_bar_bounds = pd.DataFrame(error_bar_bounds)

print(error_bar_bounds)

In [ ]:
def add_significance_indicator(ax, x1, x2, y, height, significance):
    ax.plot([x1, x1, x2, x2], [y, y+height, y+height, y], lw=1.5, c='black')
    ax.text((x1+x2)*0.5, y+height*1.5, significance, ha='center', va='bottom', color='black')

group_names = grouped_df.index.tolist()
group_indices = np.arange(len(group_names))

def get_error_bars(group, measure):
    lower = error_bar_bounds[(error_bar_bounds['Group'] == group) & (error_bar_bounds['Measure'] == 'Lower Bound')][measure].values[0]
    upper = error_bar_bounds[(error_bar_bounds['Group'] == group) & (error_bar_bounds['Measure'] == 'Upper Bound')][measure].values[0]
    mean = grouped_df.loc[group][measure]
    return [mean - lower, upper - mean]

fig, ax = plt.subplots(2, 1, figsize=(10, 8))

cell_count_errors = [get_error_bars(group, 'Cell Count') for group in group_names]
ax[0].bar(group_indices, grouped_df['Cell Count'], yerr=np.transpose(cell_count_errors), color='skyblue', capsize=5)
ax[0].set_xticks(group_indices)
ax[0].set_xticklabels(group_names)
ax[0].set_ylabel('Cell Count')
ax[0].set_title('Cell Count by Group')

intensity_errors = [get_error_bars(group, 'Normalized Intensity by Day 1') for group in group_names]
ax[1].bar(group_indices, grouped_df['Normalized Intensity by Day 1'], yerr=np.transpose(intensity_errors), color='lightgreen', capsize=5)
ax[1].set_xticks(group_indices)
ax[1].set_xticklabels(group_names)
ax[1].set_ylabel('Normalized Intensity')
ax[1].set_title('Fluorescence intensity')

for _, row in t_test_results_df.iterrows():
    groups_test = row['Group Pair'].replace(' (Cell Count)', '').replace(' (Normalized Intensity)', '')
    groups = groups_test.split(' vs ')

    try:
        x1 = group_names.index(groups[0].strip())
        x2 = group_names.index(groups[1].strip())

        if 'Cell Count' in row['Group Pair']:
            y = max(grouped_df.loc[groups[0]]['Cell Count'], grouped_df.loc[groups[1]]['Cell Count']) + 5
            stars = '*' if row['P-Value'] < 0.05 else 'ns'
            add_significance_indicator(ax[0], x1, x2, y, 5, stars)
        elif 'Intensity' in row['Group Pair']:
            y = max(grouped_df.loc[groups[0]]['Normalized Intensity by Day 1'], grouped_df.loc[groups[1]]['Normalized Intensity by Day 1']) + 0.05
            stars = '*' if row['P-Value'] < 0.05 else 'ns'
            add_significance_indicator(ax[1], x1, x2, y, 0.05, stars)
    except ValueError:
        print(f"Error processing group pair: {groups}")

plt.tight_layout()
plt.show()